# Interactive Heatmaps With Clustergrammer2
Author: Jon Katz  
Date: 2021-10-25  
Description: Example interactive heatmap using PDX expression data  

Steps:  

    1. Install interactive widgets  
    2. Request expression data  
    3. Shape expression data  
    4. Create Primary Expression Matrix  
    5. Create metadata DataFrame  
    6. Create heatmap  



## 1. Install Interactive Widgets
`jupyter_contrib_nbextensions` allows the clustergrammer2 widget to display a JavaScript widget in the notebook.  
See https://github.com/ismms-himc/clustergrammer2/issues/82, https://github.com/Jupyter-contrib/jupyter_nbextensions_configurator/issues/37

In [29]:
import numpy as np
import pandas as pd
!pip install clustergrammer2
from clustergrammer2 import net, Network, CGM2
!pip install jupyter_contrib_nbextensions
!jupyter contrib nbextension install --user
!jupyter nbextension enable --py clustergrammer2
import warnings
import requests
warnings.filterwarnings('ignore')

[I 20:37:10 InstallContribNbextensionsApp] jupyter contrib nbextension install --user
[I 20:37:10 InstallContribNbextensionsApp] Installing jupyter_contrib_nbextensions nbextension files to jupyter data directory
[I 20:37:10 InstallContribNbextensionsApp] Installing /opt/conda/lib/python3.9/site-packages/jupyter_contrib_nbextensions/nbextensions/addbefore -> addbefore
[I 20:37:10 InstallContribNbextensionsApp] Up to date: /home/jovyan/.local/share/jupyter/nbextensions/addbefore/addbefore.yaml
[I 20:37:10 InstallContribNbextensionsApp] Up to date: /home/jovyan/.local/share/jupyter/nbextensions/addbefore/icon.png
[I 20:37:10 InstallContribNbextensionsApp] Up to date: /home/jovyan/.local/share/jupyter/nbextensions/addbefore/main.js
[I 20:37:10 InstallContribNbextensionsApp] Up to date: /home/jovyan/.local/share/jupyter/nbextensions/addbefore/readme.md
[I 20:37:10 InstallContribNbextensionsApp] - Validating: OK
[I 20:37:10 InstallContribNbextensionsApp] Installing /opt/conda/lib/python3.9/

## 2. Request Expression Data
To run this block you need to replace the request_client, request_user, request_workspace_id, and request_key with your own values.

In [4]:
gene_list = ["AKT1", "BRCA1", "BRCA2", "EGFR"]
data = {
    "request_data_type": "expression",
    "request_cancer_type": ["all"],
    "request_genes": gene_list,
    "request_models": [],
    "request_dataset": "PDX",
    "request_key": "92b0532949f64b5aa88f8362f066e69e92b0532949f64b5aa88f8362f066e69e",
    "request_client": 99,
    "request_user": 4326,
    "request_workspace_id": "4326-99",
    "request_mode": 'true'
}
response = requests.post('https://lumin-fast-api.championsoncology.com/WS', json=data)
print(response.json())

{'meta': {'name': 'requested_data---25-10-2021---20:01:01 chunk-0.json', 'path': 'requested_data---25-10-2021---20:01:01 chunk-0.json', 'last_modified': '2021-10-25T20:01:08.829963Z', 'created': '2021-10-25T20:01:08.829963Z', 'content': None, 'format': None, 'mimetype': 'application/json', 'size': 1481121, 'writable': True, 'type': 'file'}}


In [5]:
datpath = response.json()['meta']['name']
# or hardcode the path
# datpath = 'requested_data---25-10-2021---20:01:01 chunk-0.json'
df = pd.read_json('../' + datpath)

## 3. Shape Expression Data 

In [7]:
df = df.drop(columns=['v1', 'gene_id', 'log.rsem.rpkm', 'log.tpm', 'fold', 'trans', 'model', 'sort_key', 'charge'])

## 4. Create Primary Expression Matrix

In [8]:
gene_df = df.pivot(index='gene', columns='model_name', values='z')\
    .dropna(1)\
    .apply(pd.to_numeric)

print(gene_df)

model_name  CTG-0009  CTG-0011  CTG-0012  CTG-0017  CTG-0018  CTG-0019  \
gene                                                                     
AKT1       -1.773188 -0.521731  0.941586  2.388352  0.672424 -1.027166   
BRCA1      -1.232682  0.366787 -1.682502 -0.751532 -0.789899 -0.081842   
BRCA2      -0.519620 -1.556504 -0.303266 -0.318489 -1.080384 -0.144858   
EGFR        0.624262  0.066407 -0.369141 -0.779709  0.182228 -0.536679   

model_name  CTG-0033  CTG-0052  CTG-0058  CTG-0059  ...  CTG-3604  CTG-3610  \
gene                                                ...                       
AKT1        2.674566  2.093480 -0.434627  0.517075  ...  1.645270  0.197581   
BRCA1      -1.860533 -1.761792 -0.798799  0.207843  ... -0.060498  0.448929   
BRCA2       0.366260 -1.809702 -0.486335 -0.912703  ... -0.211208 -2.539690   
EGFR       -0.998649 -0.063161 -0.485695 -0.416754  ... -0.051525 -1.010120   

model_name  CTG-3615  CTG-3628  CTG-3629  CTG-3651  CTG-3710  CTG-3713  \
gene  

## 5. Create Metadata DataFrame

In [9]:
type_df = df[['model_name', 'tumor_type']]\
    .reset_index(drop=True)\
    .groupby(['model_name', 'tumor_type'])\
    .agg(lambda x: x.iloc[0])[['tumor_type']]

type_df.index = type_df.index.droplevel(1)

print(type_df)

                          tumor_type
model_name                          
CTG-0009    Adenoid cystic carcinoma
CTG-0011          Cholangiocarcinoma
CTG-0012                      Breast
CTG-0017                      Breast
CTG-0018                      Breast
...                              ...
CTG-3802                    Lymphoma
CTG-3803                    Lymphoma
CTG-3804                    Lymphoma
CTG-3805                    Lymphoma
CTG-3847                    Lymphoma

[1090 rows x 1 columns]


## 6. Create Heatmap
If the heatmap does not appear, make sure you have run the first block of code to enable the clustergrammer2 widget. Then you may need to restart your kernel, or log out and log in again.

In [31]:
net.load_df(gene_df, meta_col=type_df)
net.cluster()
net.set_manual_category(col='tumor_type')
net.widget()

CGM2(manual_cat='{"col": {"col_cat_colors": {"tumor_type: Adenoid cystic carcinoma": "#393b79", "tumor_type: A…